# Designing a FIR filter with a window

In this notebook we are going to look at how to design a FIR filter using a window.
Do the imports

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.signal as signal
plt.rcParams['figure.figsize'] = (6.0, 3)

Design the window, here a square window for $H(m)$

In [ ]:
N = 64

# Initialising  an array to hold H
H = np.zeros(N)

# Set the one sided length (in m) of the filter
K2 = 5 # K/2+1

# Write this on both sides of the H (remembering the symmetry). The DC-component
# (m=0) is not written.
H[0:K2+1] = 1
H[N-K2:N] = 1


# Plot the resulting frequency response as a function of m
plt.figure(1)
plt.clf()
plt.plot(H,'o')
plt.xlabel('$m$')
plt.ylabel('$H(m)$')
plt.grid()
plt.tight_layout()
plt.show()

# Centering on 0. 
# Make the m's go from -N/2+1 to N/2
m = np.arange(-int(N/2)+1,int(N/2)+1,1)


# Plotting the new figure 
plt.figure(2)
plt.clf()
Hs = np.zeros(N)
Hs[int(N/2)-1:] = H[0:int(N/2)+1]
Hs[0:int(N/2)-1]= H[int(N/2)+1:]
plt.plot(m,Hs,'o')
plt.xlabel('$m$')
plt.ylabel('$H(m)$')
plt.grid()
plt.tight_layout()
plt.show()
    

Notice that the x-axis is in $m$, and not in frequency, meaning that the cut-off
frequency will depend on the sampling frequency.

### Calculating the filter coefficients

We are now going to calculate the filter coefficients by doing the invers DFT
$$
h(k) = \frac{1}{N}\sum_{m=-(N/2)+1}^{N/2} H(m) e^{j 2\pi m k /N}
$$

In [ ]:
# Finding the filter coefficients by taking the inverse FFT
ht = np.fft.ifft(H,N)

# We are looking for the real values
ht = np.real(ht)
# Plot the figure of the filter coefficients 
plt.figure(3)
plt.clf()
plt.plot(ht,'o')
plt.xlabel('$k$')
plt.ylabel('$h[k]$')
plt.grid()
plt.tight_layout()
plt.show()



In the figure we need to check that the filter coefficients are symetrical and 
don't wary to much. 

In [ ]:
# We want to shift it to the center to make the final filter symmetric

h = np.roll(ht,32) # Roll the filter 32 steps 

h=h[1:] # Removing the point making it non-symetric 
plt.figure(31)
plt.clf()
plt.plot(h,'o')
plt.xlabel('$k$')
plt.ylabel('$h[k]$')
plt.grid()
plt.tight_layout()
plt.show()


Once you are sattisfied about this we will make the signal to test how the filter 
works. 


In [ ]:
N2 = 100 
n = np.arange(N2)

fs = 50 # Sampling frequency in Hz
ts = 1/fs # Sampling time in s 

f1 = 1 # Frequency of the first signal in Hz
f2 = 20 # Frequency of the second signal in Hz

# Making the signal as a sum of two cosines. 
x = np.cos(2*np.pi*f1*n*ts)+np.cos(2*np.pi*f2*n*ts)

# Plot the resulting input signal in the time domain
plt.figure(4)
plt.clf()
plt.plot(n,x,'-o')
plt.xlabel('$n$')
plt.ylabel('$x[n]$')
plt.title('Input signal')
plt.grid()
plt.tight_layout()
plt.show()

# Take the FFT of the input signal and plot it 
plt.figure(5)
plt.clf()
plt.plot(n*fs/N,np.abs(np.fft.fft(x)),'-o')
plt.xlabel('$f (Hz)$')
plt.ylabel('$X$')
plt.title('FFT of input signal')
plt.grid()
plt.tight_layout()
plt.show()

# Apply the filter to the input signal using convolution 
y = np.convolve(h,x)

# Plot the resulting output signal 
plt.figure(6)
plt.clf()
plt.plot(y,'-o')
plt.xlabel('$n$')
plt.ylabel('$y[n]$')
plt.title('Output signal')
plt.grid()
plt.tight_layout()
plt.show()

# Plot the FFT of the signal 
plt.figure(7)
plt.clf()
plt.plot(np.arange(len(y))*fs/len(y),np.abs(np.fft.fft(y)),'-o')
plt.xlabel('$f(Hz)$')
plt.ylabel('$Y_{mag}$')
plt.title('FFT of output signal')
plt.grid()
plt.tight_layout()
plt.show()



## A filter design using a window 

We will now use the *scipy* method of [firwin](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html#scipy.signal.firwin) to generate a filter.

We will first take a look at the window and see how it looks. The window of choice is the Dolph-Chebyshev window.

In [ ]:

# Make the window, by setting the number of coefficients, M, and the attenu
Nx = 64 # Number of coefficients 
at = 100 # Attenuation of the window in dB

# Get the window. 
window = signal.get_window(('chebwin', at), Nx) 

# Plot the resulting window 
plt.figure(8)
plt.clf()
plt.plot(window)
plt.title("Dolph-Chebyshev window (100 dB)")
plt.ylabel("$H[m]$")
plt.xlabel("$m$")

# Find the resulting filter coefficients
hw = np.fft.ifft(window,Nx)
# Plot the magnitude of the inverse FFT of the window
plt.figure(9)
plt.clf()
plt.plot(np.abs(hw),'.')
plt.show()
#plt.title("Frequency response of the Dolph-Chebyshev window (100 dB)")
#plt.ylabel("Normalized magnitude [dB]")
#plt.xlabel("Normalized frequency [cycles per sample]")

In [ ]:
hww = np.roll(hw,32) # Roll the filter 32 steps 

hww = hww[1:] # Removing the point making it non-symetric 
plt.figure(10)
plt.clf()
plt.plot(hww,'o')
plt.xlabel('$k$')
plt.ylabel('$h[k]$')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Apply the filter to the input signal using convolution 
y = np.convolve(hww,x)

# Plot the resulting output signal 
plt.figure(11)
plt.clf()
plt.plot(y,'-o')
plt.xlabel('$n$')
plt.ylabel('$y[n]$')
plt.title('Output signal')
plt.grid()
plt.tight_layout()
plt.show()

# Plot the FFT of the signal 
plt.figure(12)
plt.clf()
plt.plot(np.arange(len(y))*fs/len(y),np.abs(np.fft.fft(y)),'-o')
plt.xlabel('$f(Hz)$')
plt.ylabel('$Y_{mag}$')
plt.title('FFT of output signal')
plt.grid()
plt.tight_layout()
plt.show()